In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df19 = df19.apply(pd.to_numeric, errors = 'coerce')
df19.describe().iloc[:, :14]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal
count,9.634800e+04,96237.000000,96237.000000,95682.000000,94572.000000,95127.000000,92130.000000,95682.000000,95682.000000,95238.000000,95238.000000,0.0,96237.000000,96237.000000
mean,1.366763e+18,58.394464,42.025375,50.494200,1.416667,39.529755,45.286747,16.704176,2.198376,544.264569,1790.674825,NaN,0.101153,0.073870
std,2.634214e+16,20.272105,18.383286,19.070779,8.618257,18.125533,17.081886,16.386358,4.662293,103.844503,107.791752,NaN,0.540373,0.208515
min,1.325376e+18,-4.000000,-14.000000,-8.000000,-20.000000,-20.000000,-10.000000,0.000000,0.000000,412.000000,1617.000000,NaN,0.000000,0.000000
25%,1.344103e+18,42.000000,29.000000,36.000000,-5.000000,26.000000,33.000000,0.000000,0.000000,439.000000,1709.000000,NaN,0.000000,0.000000
50%,1.362917e+18,60.000000,43.000000,52.000000,0.000000,41.000000,47.000000,13.000000,0.000000,534.000000,1809.000000,NaN,0.000000,0.000000
75%,1.389074e+18,76.000000,57.000000,66.000000,7.000000,54.000000,59.000000,29.000000,1.000000,631.000000,1909.000000,NaN,0.000000,0.020000
max,1.414714e+18,103.000000,81.000000,92.000000,33.000000,73.000000,76.000000,73.000000,27.000000,723.000000,1934.000000,NaN,6.100000,2.450000


In [3]:
df19['store_nbr'].unique()

array([30], dtype=int64)

In [5]:
df19_drop_columns = ['date', 'codesum', 'station_nbr', 'store_nbr']
df19 = df19.drop(columns = df19_drop_columns)

In [6]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df19_columns = df19.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df19_columns:
    if (i == 'resultdir'):
        df19[i].fillna(df19[i].mode()[0], inplace=True)
        print(df19[i].mode()[0])
    else:
        df19[i].fillna(df19[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df19['relative_humility'] = 100*(np.exp((19.625*((df19['dewpoint']-32)/1.8))/(243.04+((df19['dewpoint']-32)/1.8)))/np.exp((19.625*((df19['tavg']-32)/1.8))/(243.04+((df19['tavg']-32)/1.8))))

# 체감온도 계산
df19["windchill"] = 35.74 + 0.6219*df19["tavg"] - 35.75*(df19["avgspeed"]**0.19) + 0.4275*df19["tavg"]*(df19["avgspeed"]**0.19)

df19 = df19[df19['units'] != 0]

2


In [9]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df19 = sm.OLS.from_formula('np.log1p(units) ~ depart + sunset + sunrise + tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df19)
result_df19 = model_df19.fit()

print(result_df19.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     865.3
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:04:45   Log-Likelihood:                -1705.2
No. Observations:                2447   AIC:                             3472.
Df Residuals:                    2416   BIC:                             3652.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]         0.0242      0.04

In [12]:
model_df19 = sm.OLS.from_formula('np.log1p(units) ~ scale(depart) + scale(sunset) + scale(sunrise) + scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df19)
result_df19 = model_df19.fit()

print(result_df19.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     865.3
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:05:42   Log-Likelihood:                -1705.2
No. Observations:                2447   AIC:                             3472.
Df Residuals:                    2416   BIC:                             3652.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [13]:
anova_result_df19 = sm.stats.anova_lm(result_df19).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df19[anova_result_df19['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(preciptotal),1.0,0.998782,0.998782,4.179610,4.102131e-02
C(rain),1.0,1.271513,1.271513,5.320912,2.115520e-02
scale(heat),1.0,1.300453,1.300453,5.442015,1.973968e-02
C(other),1.0,1.720270,1.720270,7.198827,7.344855e-03
scale(relative_humility),1.0,1.821892,1.821892,7.624088,5.802909e-03
scale(dewpoint),1.0,1.876659,1.876659,7.853269,5.113340e-03
scale(resultdir),1.0,2.025624,2.025624,8.476645,3.630301e-03
scale(resultspeed),1.0,2.771092,2.771092,11.596210,6.716601e-04
scale(snowfall),1.0,3.524888,3.524888,14.750629,1.258633e-04
scale(depart),1.0,6.916108,6.916108,28.941892,8.175822e-08


In [14]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df19.values, i) for i in range(df19.shape[1])]
vif["features"] = df19.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# scale(preciptotal) + C(other) + scale(relative_humility) + scale(resultdir) + scale(snowfall) + scale(depart) + C(week7) + C(item_nbr)

,VIF Factor,features
0,1.056652,item_nbr
1,1.060158,units
2,1.373317,preciptotal
3,1.472131,resultdir
4,1.596777,other
5,1.811263,snowfall
6,2.586164,depart
7,2.784719,weekend
8,2.798713,week7
9,5.247424,relative_humility


In [15]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df19 = sm.OLS.from_formula('np.log1p(units) ~ scale(preciptotal) + C(other) + scale(relative_humility) + scale(resultdir) + scale(snowfall) + scale(depart) + C(week7) + C(item_nbr) + 0', data = df19)
result_df19 = model_df19.fit()

print(result_df19.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     1408.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:07:57   Log-Likelihood:                -1801.8
No. Observations:                2447   AIC:                             3640.
Df Residuals:                    2429   BIC:                             3744.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(other)[0]             

In [17]:
# scale(preciptotal) + C(other) + scale(relative_humility) + scale(resultdir) + scale(snowfall) + scale(depart) + C(week7) + C(item_nbr)
X19 = df19[['preciptotal', 'other', 'relative_humility', 'resultdir', 'snowfall', 'depart', 'week7', 'item_nbr']]
y19 = df19['units']
model19 = LinearRegression()

cv19 = KFold(n_splits=19, shuffle=True, random_state=0)

cross_val_score(model19, X19, y19, scoring="r2", cv=cv19)

array([ 0.08896858, -0.00173713,  0.03031222,  0.09929822,  0.0215694 ,
        0.03435218, -0.11264308,  0.03585828,  0.03484771,  0.01600532,
       -0.06649595,  0.03830329,  0.07579344,  0.04982989,  0.049533  ,
        0.04651345, -0.07843362, -0.02451548,  0.0384792 ])